# PARTIE 1 : Nettoyage des données

## Import librairies & création session Spark

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Lecture des données

In [0]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"
df = spark.read.format('json').load(filepath, multiline=True)
print(type(df))
print(df.count(), len(df.columns))

<class 'pyspark.sql.dataframe.DataFrame'>
55691 2


In [0]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

## Nettoyage des champs

Les caractères spéciaux sont remplacés par underscore "_"  
Les caractères "&" sont remplacés par "And"  
La structure des données ci-dessus montre l'existence de champs imbriqués dans d'autres champs.  
La fonction ci-dessous permet de descendre et parcourir tous les champs imbriqués pour les nettoyer.

#### Fonction de nettoyage

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.types import DataType, StructType, StructField, ArrayType
from copy import copy

def cleanDataFrame(df: DataFrame) -> DataFrame:
    ''' Fonction qui prend en paramètre un dataframe et retourne un dataframe nettoyé '''

    def sanitizeFieldName(s: str) -> str:
        ''' Fonction qui remplace les caractères spéciaux par underscore ou And '''
        return s.replace("&", "And").replace("-", "_").replace("\"", "_")\
            .replace("[", "_").replace("]", "_").replace(".", "_")\
            .replace(" ", "_").replace("'", "")
    
    def sanitizeField(field: StructField) -> StructField:
        ''' Fonction qui applique la fonction sanitizeFieldName() sur les champs '''
        field = copy(field)
        field.name = sanitizeFieldName(field.name)
        field.dataType = cleanSchema(field.dataType)
        return field
    
    def cleanSchema(dataType: [DataType]) -> [DataType]:
        ''' Fonction qui permet de descendre dans tous les champs imbriqués pour y appliquer la fonction de nettoyage '''
        dataType = copy(dataType)
        if isinstance(dataType, StructType):
            dataType.fields = [sanitizeField(f) for f in dataType.fields]
        elif isinstance(dataType, ArrayType):
            dataType.elementType = cleanSchema(dataType.elementType)
        return dataType

    # La fonction retourne un nouveau dataframe créé à partir du schéma nettoyé
    return spark.createDataFrame(df.rdd, cleanSchema(df.schema))

#### Application

In [0]:
df = cleanDataFrame(df)

Les noms des champs ont bien été nettoyées :

In [0]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

## Récupération des champs

#### Fonction pour récupérer les champs

In [0]:
from pyspark.sql.types import StructType, StructField
from typing import List, Dict, Generator, Union, Callable

def walkSchema(schema: Union[StructType, StructField]) -> Generator[str, None, None]:
    """ Fonction qui parcourt le schéma pour retourner tous les champs et champs imbriqués """
    """ Elle prend en paramètre un schéma PySpark et retourne un Generator qui est un itérable """

    def _walk(schema_dct: Dict['str', Union['str', list, dict]],
              prefix: str = "") -> Generator[str, None, None]:
        assert isinstance(prefix, str), "prefix should be a string"
        
        fullName: Callable[str, str] = lambda name: ( 
            name if not prefix else f"{prefix}.{name}")
        
        name = schema_dct.get('name', '')

        if schema_dct['type'] == 'struct':
            assert 'fields' in schema_dct, (
                "It's a StructType, we should have some fields")
            for field in schema_dct['fields']:
                yield from _walk(field, prefix=prefix)
        elif isinstance(schema_dct['type'], dict):
            assert 'fields' not in schema_dct, (
                "We're missing some keys here")
            yield from _walk(schema_dct['type'], prefix=fullName(name))
        elif name:
            yield fullName(name)
    
    yield from _walk(schema.jsonValue())

#### Application

In [0]:
col_names = walkSchema(df.schema)

In [0]:
for col in col_names:
    print(col)

data.appid
data.ccu
data.developer
data.discount
data.genre
data.header_image
data.initialprice
data.languages
data.name
data.negative
data.owners
data.platforms.linux
data.platforms.mac
data.platforms.windows
data.positive
data.price
data.publisher
data.release_date
data.required_age
data.short_description
data.tags.1980s
data.tags.1990s
data.tags.2_5D
data.tags.2D
data.tags.2D_Fighter
data.tags.2D_Platformer
data.tags.360_Video
data.tags.3D
data.tags.3D_Fighter
data.tags.3D_Platformer
data.tags.3D_Vision
data.tags.4_Player_Local
data.tags.4X
data.tags.6DOF
data.tags.8_bit_Music
data.tags.ATV
data.tags.Abstract
data.tags.Action
data.tags.Action_RPG
data.tags.Action_RTS
data.tags.Action_Roguelike
data.tags.Action_Adventure
data.tags.Addictive
data.tags.Adventure
data.tags.Agriculture
data.tags.Aliens
data.tags.Alternate_History
data.tags.Ambient
data.tags.America
data.tags.Animation_And_Modeling
data.tags.Anime
data.tags.Arcade
data.tags.Archery
data.tags.Arena_Shooter
data.tags.Artifi

## Éclatement en plusieurs colonnes

Nous utilisons la méthode reduce() pour éclater les champs en plusieurs colonnes.  
La colonne *data* devient inutile, nous la retirons. Nous retirons aussi la colonne *id* qui est un doublon de *appid*


In [0]:
from functools import reduce
from pyspark.sql import functions as F

exploded_df = reduce(
  lambda memo_df, col_name: memo_df.withColumn(col_name, F.col(col_name)),
  walkSchema(df.schema), df
).drop('data', 'id') 

In [0]:
exploded_df.printSchema()

root
 |-- data.appid: long (nullable = true)
 |-- data.ccu: long (nullable = true)
 |-- data.developer: string (nullable = true)
 |-- data.discount: string (nullable = true)
 |-- data.genre: string (nullable = true)
 |-- data.header_image: string (nullable = true)
 |-- data.initialprice: string (nullable = true)
 |-- data.languages: string (nullable = true)
 |-- data.name: string (nullable = true)
 |-- data.negative: long (nullable = true)
 |-- data.owners: string (nullable = true)
 |-- data.platforms.linux: boolean (nullable = true)
 |-- data.platforms.mac: boolean (nullable = true)
 |-- data.platforms.windows: boolean (nullable = true)
 |-- data.positive: long (nullable = true)
 |-- data.price: string (nullable = true)
 |-- data.publisher: string (nullable = true)
 |-- data.release_date: string (nullable = true)
 |-- data.required_age: string (nullable = true)
 |-- data.short_description: string (nullable = true)
 |-- data.tags.1980s: long (nullable = true)
 |-- data.tags.1990s: long

## Suppression du mot "data" et des points

Nous pouvons supprimer le mot "data" pour simplifier le nom des colonnes.  
Les noms des colonnes contiennent des points.  
Nous devons les enlever car ils génèrent de l'ambiguité quand il s'agit de les sélectionner une colonne dans un dataframe.

#### Fonction pour enlever le mot "data" et les points

In [0]:
def sanitize_column_name(col_name): 
  """ Fonction qui ne garde que la partie après le premier point """
  """ Et qui remplace les points suivants par underscore """
  return '_'.join(col_name.split(".")[1:])

#### Application

In [0]:
from pyspark.sql import functions as F 
from functools import reduce

df_renamed = reduce(
  lambda memo_df, col_name: memo_df.withColumnRenamed(
    col_name, sanitize_column_name(col_name)),
  exploded_df.columns, exploded_df
)

In [0]:
df_renamed.printSchema()

root
 |-- appid: long (nullable = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms_linux: boolean (nullable = true)
 |-- platforms_mac: boolean (nullable = true)
 |-- platforms_windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- tags_1980s: long (nullable = true)
 |-- tags_1990s: long (nullable = true)
 |-- tags_2_5D: long (nullable = true)
 |-- tags_2D: long (nullable = true)
 |-- tags_2D_Fi

In [0]:
df_renamed.columns

Out[41]: ['appid',
 'ccu',
 'developer',
 'discount',
 'genre',
 'header_image',
 'initialprice',
 'languages',
 'name',
 'negative',
 'owners',
 'platforms_linux',
 'platforms_mac',
 'platforms_windows',
 'positive',
 'price',
 'publisher',
 'release_date',
 'required_age',
 'short_description',
 'tags_1980s',
 'tags_1990s',
 'tags_2_5D',
 'tags_2D',
 'tags_2D_Fighter',
 'tags_2D_Platformer',
 'tags_360_Video',
 'tags_3D',
 'tags_3D_Fighter',
 'tags_3D_Platformer',
 'tags_3D_Vision',
 'tags_4_Player_Local',
 'tags_4X',
 'tags_6DOF',
 'tags_8_bit_Music',
 'tags_ATV',
 'tags_Abstract',
 'tags_Action',
 'tags_Action_RPG',
 'tags_Action_RTS',
 'tags_Action_Roguelike',
 'tags_Action_Adventure',
 'tags_Addictive',
 'tags_Adventure',
 'tags_Agriculture',
 'tags_Aliens',
 'tags_Alternate_History',
 'tags_Ambient',
 'tags_America',
 'tags_Animation_And_Modeling',
 'tags_Anime',
 'tags_Arcade',
 'tags_Archery',
 'tags_Arena_Shooter',
 'tags_Artificial_Intelligence',
 'tags_Assassin',
 'tags_Asy

In [0]:
len(df_renamed.columns)

Out[42]: 463

## Suppression des colonnes inutiles

Le dataframe contient plus de 460 colonnes. Nous identifions les colonnes inutiles pour notre étude.

In [0]:
df_renamed.select('type').groupBy('type').count().orderBy(F.desc('count')).show()

+--------+-----+
|    type|count|
+--------+-----+
|    game|55690|
|hardware|    1|
+--------+-----+



Nous remarquons que toutes les valeurs de la colonne *type* sont *game* excepté pour l'application Steam Link qui est un *hardware*.  
Nous pouvons supprimer cette application de notre jeu de données.

In [0]:
df_renamed.createOrReplaceTempView('steam_games_table')
display(spark.sql("""
    SELECT * FROM steam_games_table
    WHERE type = 'hardware'
"""))

appid,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms_linux,platforms_mac,platforms_windows,positive,price,publisher,release_date,required_age,short_description,tags_1980s,tags_1990s,tags_2_5D,tags_2D,tags_2D_Fighter,tags_2D_Platformer,tags_360_Video,tags_3D,tags_3D_Fighter,tags_3D_Platformer,tags_3D_Vision,tags_4_Player_Local,tags_4X,tags_6DOF,tags_8_bit_Music,tags_ATV,tags_Abstract,tags_Action,tags_Action_RPG,tags_Action_RTS,tags_Action_Roguelike,tags_Action_Adventure,tags_Addictive,tags_Adventure,tags_Agriculture,tags_Aliens,tags_Alternate_History,tags_Ambient,tags_America,tags_Animation_And_Modeling,tags_Anime,tags_Arcade,tags_Archery,tags_Arena_Shooter,tags_Artificial_Intelligence,tags_Assassin,tags_Asymmetric_VR,tags_Asynchronous_Multiplayer,tags_Atmospheric,tags_Audio_Production,tags_Auto_Battler,tags_Automation,tags_Automobile_Sim,tags_BMX,tags_Base_Building,tags_Baseball,tags_Based_On_A_Novel,tags_Basketball,tags_Battle_Royale,tags_Beat_em_up,tags_Beautiful,tags_Benchmark,tags_Bikes,tags_Blood,tags_Board_Game,tags_Boss_Rush,tags_Bowling,tags_Boxing,tags_Building,tags_Bullet_Hell,tags_Bullet_Time,tags_CRPG,tags_Capitalism,tags_Card_Battler,tags_Card_Game,tags_Cartoon,tags_Cartoony,tags_Casual,tags_Cats,tags_Character_Action_Game,tags_Character_Customization,tags_Chess,tags_Choices_Matter,tags_Choose_Your_Own_Adventure,tags_Cinematic,tags_City_Builder,tags_Class_Based,tags_Classic,tags_Clicker,tags_Co_op,tags_Co_op_Campaign,tags_Coding,tags_Cold_War,tags_Collectathon,tags_Colony_Sim,tags_Colorful,tags_Combat,tags_Combat_Racing,tags_Comedy,tags_Comic_Book,tags_Competitive,tags_Conspiracy,tags_Controller,tags_Conversation,tags_Cooking,tags_Cozy,tags_Crafting,tags_Creature_Collector,tags_Cricket,tags_Crime,tags_Crowdfunded,tags_Cult_Classic,tags_Cute,tags_Cyberpunk,tags_Cycling,tags_Dark,tags_Dark_Comedy,tags_Dark_Fantasy,tags_Dark_Humor,tags_Dating_Sim,tags_Deckbuilding,tags_Demons,tags_Design_And_Illustration,tags_Destruction,tags_Detective,tags_Difficult,tags_Dinosaurs,tags_Diplomacy,tags_Documentary,tags_Dog,tags_Dragons,tags_Drama,tags_Driving,tags_Dungeon_Crawler,tags_Dungeons_And_Dragons,tags_Dynamic_Narration,tags_Dystopian_,tags_Early_Access,tags_Economy,tags_Education,tags_Electronic,tags_Electronic_Music,tags_Emotional,tags_Epic,tags_Episodic,tags_Escape_Room,tags_Experience,tags_Experimental,tags_Exploration,tags_FMV,tags_FPS,tags_Faith,tags_Family_Friendly,tags_Fantasy,tags_Farming,tags_Farming_Sim,tags_Fast_Paced,tags_Feature_Film,tags_Female_Protagonist,tags_Fighting,tags_First_Person,tags_Fishing,tags_Flight,tags_Football,tags_Foreign,tags_Free_to_Play,tags_Funny,tags_Futuristic,tags_Gambling,tags_Game_Development,tags_GameMaker,tags_Games_Workshop,tags_Gaming,tags_God_Game,tags_Golf,tags_Gore,tags_Gothic,tags_Grand_Strategy,tags_Great_Soundtrack,tags_Grid_Based_Movement,tags_Gun_Customization,tags_Hack_and_Slash,tags_Hacking,tags_Hand_drawn,tags_Hardware,tags_Heist,tags_Hentai,tags_Hero_Shooter,tags_Hex_Grid,tags_Hidden_Object,tags_Historical,tags_Hockey,tags_Horror,tags_Horses,tags_Hunting,tags_Idler,tags_Illuminati,tags_Immersive,tags_Immersive_Sim,tags_Indie,tags_Instrumental_Music,tags_Intentionally_Awkward_Controls,tags_Interactive_Fiction,tags_Inventory_Management,tags_Investigation,tags_Isometric,tags_JRPG,tags_Jet,tags_Job_Simulator,tags_Jump_Scare,tags_Kickstarter,tags_LEGO,tags_LGBTQ+,tags_Lemmings,tags_Level_Editor,tags_Life_Sim,tags_Linear,tags_Local_Co_Op,tags_Local_Multiplayer,tags_Logic,tags_Loot,tags_Looter_Shooter,tags_Lore_Rich,tags_Lovecraftian,tags_MMORPG,tags_MOBA,tags_Magic,tags_Mahjong,tags_Management,tags_Mars,tags_Martial_Arts,tags_Massively_Multiplayer,tags_Masterpiece,tags_Match_3,tags_Mature,tags_Mechs,tags_Medical_Sim,tags_Medieval,tags_Memes,tags_Metroidvania,tags_Military,tags_Mini_Golf,tags_Minigames,tags_Minimalist,tags_Mining,tags_Mod,tags_Moddable,tags_Modern,tags_Motocross,tags_Motorbike,tags_Mouse_only,tags_Movie,tags_Mult

In [0]:
useless_cols = ['header_image', 'short_description', 'type', 'website']
df_clean = df_renamed.alias('df_clean')
df_clean = df_clean.filter(df_clean['type'] != 'hardware')
df_clean = df_clean.drop(*useless_cols)

## Mettre le prix au bon format

Nous mettons les colonnes *price*, *initialprice* et *discount* au format float et integer.  
Nous divisons les colonnes *price* et *initalprice* par 100.

In [0]:
from pyspark.sql.types import IntegerType
df_clean = df_clean \
    .withColumn('price', df_renamed['price'].cast('float')) \
    .withColumn('initialprice', df_renamed['initialprice'].cast('float')) \
    .withColumn('discount', df_renamed['discount'].cast(IntegerType()))


In [0]:
df_clean = df_clean \
    .withColumn('price', F.col('price')/100) \
    .withColumn('initialprice', F.col('initialprice')/100)

# PARTIE 2 : Analyse des données

## Analyse au niveau macro

#### 1. Quel éditeur a sorti le plus de jeux sur Steam ?  
--> Sega est l'éditeur qui a sorti le plus de jeux sur Steam (165)

In [0]:
df_clean.createOrReplaceTempView('steam_games_table')

spark.sql("""
          SELECT publisher, COUNT(publisher) AS cnt
          FROM steam_games_table
          GROUP BY publisher
          ORDER BY cnt DESC 
""").show()

+--------------------+---+
|           publisher|cnt|
+--------------------+---+
|      Big Fish Games|422|
|              8floor|202|
|                SEGA|165|
|      Strategy First|151|
|         Square Enix|141|
|     Choice of Games|140|
|       Sekai Project|132|
|            HH-Games|132|
|                    |132|
|             Ubisoft|127|
|        Laush Studio|126|
|          THQ Nordic|125|
|Alawar Entertainment|107|
|  Fulqrum Publishing|104|
|     Plug In Digital|101|
|            Ziggurat|100|
|     Slitherine Ltd.| 99|
|   Sokpop Collective| 99|
|    Devolver Digital| 98|
|KOEI TECMO GAMES ...| 90|
+--------------------+---+
only showing top 20 rows



#### 2. Quels sont les jeux les mieux notés ?  
--> Counter-Strike 2 avec 5.9 millions de notes positives

In [0]:
spark.sql("""
          SELECT appid, name, positive
          FROM steam_games_table
          ORDER BY positive DESC
""").show()

+-------+--------------------+--------+
|  appid|                name|positive|
+-------+--------------------+--------+
|    730|Counter-Strike: G...| 5943345|
|    570|              Dota 2| 1534895|
| 271590|  Grand Theft Auto V| 1229265|
| 578080| PUBG: BATTLEGROUNDS| 1185361|
| 105600|            Terraria| 1014711|
| 359550|Tom Clancy's Rain...|  942910|
|   4000|         Garry's Mod|  861240|
|    440|     Team Fortress 2|  846407|
| 252490|                Rust|  732513|
|    550|       Left 4 Dead 2|  643836|
| 292030|The Witcher 3: Wi...|  632627|
| 945360|            Among Us|  586302|
| 227300|Euro Truck Simula...|  572368|
| 431960|    Wallpaper Engine|  561096|
| 218620|            PAYDAY 2|  532013|
| 381210|    Dead by Daylight|  509637|
| 413150|      Stardew Valley|  497558|
| 252950|       Rocket League|  496499|
|1245620|          ELDEN RING|  490203|
| 346110|ARK: Survival Evo...|  481318|
+-------+--------------------+--------+
only showing top 20 rows



#### 3. Y a-t-il des années avec plus de sorties ? Y a-t-il eu plus ou moins de sorties de jeux pendant le Covid par exemple ?  
--> 2021, 2020, 2018 et 2022 sont les années où il y a eu le plus de jeux sortis (plus de 7000)  
--> Le record est atteint en 2021 durant le Covid : 8823 jeux sortis

In [0]:
df_release_date = df_renamed.select('release_date', F.substring('release_date',1,4).alias('year'))
df_release_date.printSchema()

root
 |-- release_date: string (nullable = true)
 |-- year: string (nullable = true)



In [0]:
df_release_date.createOrReplaceTempView('release_year_table')
spark.sql("""
          SELECT year, COUNT(year) AS cnt
          FROM release_year_table
          GROUP BY year
          ORDER BY cnt DESC
""").show()

+----+----+
|year| cnt|
+----+----+
|2021|8823|
|2020|8305|
|2018|7678|
|2022|7455|
|2019|6968|
|2017|6017|
|2016|4185|
|2015|2576|
|2014|1557|
|2013| 471|
|2012| 345|
|2009| 311|
|2010| 288|
|2011| 267|
|2008| 159|
|    |  99|
|2007|  98|
|2006|  61|
|2005|   6|
|2004|   6|
+----+----+
only showing top 20 rows



#### 4. Comment sont distribués les prix ? Y a-t-il beaucoup de jeux avec une réduction ?  
--> Les prix vont de 0 à 999 dollars  
--> Environ 2500 jeux sont en promotion soit 4% des jeux

In [0]:
df_price = df_clean.select(['initialprice', 'discount', 'price'])
df_price.printSchema()

root
 |-- initialprice: double (nullable = true)
 |-- discount: integer (nullable = true)
 |-- price: double (nullable = true)



In [0]:
from pyspark.sql.functions import col, log
df_price = df_price.withColumn("logprice", log(10.0, col("price")))

In [0]:
df_price.createOrReplaceTempView('price_table')

spark.sql("""
          SELECT initialprice
          FROM price_table
          GROUP BY initialprice
          ORDER BY initialprice DESC
""").show()

+------------+
|initialprice|
+------------+
|       999.0|
|       299.9|
|      269.99|
|       249.0|
|      199.99|
|      149.99|
|       149.0|
|      129.99|
|      124.99|
|      119.99|
|       99.99|
|        99.0|
|       94.99|
|       89.99|
|       84.99|
|       79.99|
|       74.99|
|       69.99|
|       69.95|
|       69.03|
+------------+
only showing top 20 rows



In [0]:
# Ouvrez le notebook dans Databricks pour voir la visualisation :
display(df_price.select('logprice'))

logprice
0.9995654882259822
0.9995654882259822
0.7774268223893113
1.300812794118117
0.2988530764097066
0.9025467793139913
1.113609151073028
null
0.4756711883244296
1.1458177144918276


Databricks visualization. Run in Databricks to view.

In [0]:
df_price.filter(df_price['discount'] > 0).count() 

Out[57]: 2518

In [0]:
nb_discount = df_price.filter(df_clean['discount'] > 0).count()
proportion = nb_discount/df_price.count()*100
print('proportion de discount:', proportion)

proportion de discount: 4.52145807146705


#### 5. Quelles sont les langues les plus représentées ?
--> Sur un total de 55690 jeux la langue la plus fréquente est :
1. Anglais 54646
2. Allemand 13996
3. Français 13406

In [0]:
from pyspark.sql.functions import split, col
df_languages = df_clean.select(split(col("languages"),",").alias("languages_list"))
df_languages.distinct().show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|languages_list                                                                                                                                                                                                                                                                                                                                                          |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
print(df_languages.distinct().count())

8623


In [0]:
df_languages.printSchema()

root
 |-- languages_list: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [0]:
from pyspark.sql.functions import explode
df_languages_list = df_languages.select(df_languages.languages_list, explode(df_languages.languages_list).alias('language'))
df_languages_list.show(truncate=False)
df_languages_list.count()

+-----------------------------------------------------------------------------------------------------------------------------------+--------------------+
|languages_list                                                                                                                     |language            |
+-----------------------------------------------------------------------------------------------------------------------------------+--------------------+
|[English,  French,  German,  Italian,  Spanish - Spain,  Simplified Chinese,  Traditional Chinese,  Korean]                        |English             |
|[English,  French,  German,  Italian,  Spanish - Spain,  Simplified Chinese,  Traditional Chinese,  Korean]                        | French             |
|[English,  French,  German,  Italian,  Spanish - Spain,  Simplified Chinese,  Traditional Chinese,  Korean]                        | German             |
|[English,  French,  German,  Italian,  Spanish - Spain,  Simplified C

In [0]:
df_languages_list.printSchema()

root
 |-- languages_list: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- language: string (nullable = false)



In [0]:
df_languages_list.createOrReplaceTempView('languages_table')
spark.sql("""
          SELECT language, COUNT(language) AS cnt
          FROM languages_table
          GROUP BY language
          ORDER BY cnt DESC
""").show(21, truncate=False)

+------------------------+-----+
|language                |cnt  |
+------------------------+-----+
|English                 |54646|
| German                 |13996|
| French                 |13406|
| Russian                |12839|
| Spanish - Spain        |12224|
| Simplified Chinese     |12213|
| Japanese               |10170|
| Italian                |9297 |
| Portuguese - Brazil    |6739 |
| Korean                 |6575 |
| Traditional Chinese    |6263 |
| Polish                 |5369 |
| Portuguese - Portugal  |4011 |
| Turkish                |3601 |
| Dutch                  |3076 |
| Spanish - Latin America|2729 |
| Czech                  |2340 |
| Swedish                |2047 |
| Ukrainian              |1928 |
| Hungarian              |1923 |
| Arabic                 |1794 |
+------------------------+-----+
only showing top 21 rows



#### 6. Existe-t-il de nombreux jeux interdits aux moins de 16/18 ans ?
--> Il y a 571 jeux interdits aux moins de 16 ans  
--> Sur un total de 55690 jeux cela ne représente que 1%

In [0]:
df_required_age = df_clean.select(df_clean['name'], df_clean['required_age'])
df_required_age.printSchema()

root
 |-- name: string (nullable = true)
 |-- required_age: string (nullable = true)



In [0]:
print(df_required_age.select('required_age').distinct().count())

21


In [0]:
df_required_age.createOrReplaceTempView('required_age_table')
spark.sql("""
          SELECT required_age, COUNT(required_age) as cnt
          FROM required_age_table
          GROUP BY required_age
          ORDER BY cnt DESC
""").show(21)

+------------+-----+
|required_age|  cnt|
+------------+-----+
|           0|55029|
|          15|  264|
|          18|  223|
|          16|   38|
|          17|   38|
|          12|   32|
|          13|   26|
|          14|   10|
|          10|    7|
|           6|    4|
|         180|    4|
|           3|    3|
|           8|    3|
|           7|    2|
|          35|    1|
|         21+|    1|
|           5|    1|
|           9|    1|
|          7+|    1|
|          20|    1|
|      MA 15+|    1|
+------------+-----+



Les données de la colonne *required_age* ne sont pas au format de nombre entier. Nous rectifions cela :

In [0]:
def sanitize_required_age(str):
    ''' Fonction pour remplacer les valeurs par des nombres entiers '''
    if str == 'MA 15+':
        return int(16)
    elif str == '7+':
        return int(7)
    elif str == '21+':
        return int(21)
    else:
        return int(str)

In [0]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
to_int_udf = udf(lambda x:sanitize_required_age(x),IntegerType())
df_age = df_required_age.withColumn('required_age', to_int_udf(col('required_age')))

In [0]:
df_age.createOrReplaceTempView('required_age_table')
spark.sql("""
          SELECT DISTINCT required_age
          FROM required_age_table
          ORDER BY required_age DESC
""").show(21)

+------------+
|required_age|
+------------+
|         180|
|          35|
|          21|
|          20|
|          18|
|          17|
|          16|
|          15|
|          14|
|          13|
|          12|
|          10|
|           9|
|           8|
|           7|
|           6|
|           5|
|           3|
|           0|
+------------+



In [0]:
spark.sql("""
          SELECT required_age, COUNT(required_age) as cnt
          FROM required_age_table
          WHERE required_age > 15
          GROUP BY required_age
          ORDER BY cnt DESC
""").show()

+------------+---+
|required_age|cnt|
+------------+---+
|          18|223|
|          16| 39|
|          17| 38|
|         180|  4|
|          20|  1|
|          35|  1|
|          21|  1|
+------------+---+



In [0]:
spark.sql("""
          SELECT COUNT(*) as total_prohibited_under_16
          FROM required_age_table
          WHERE required_age >= 15
""").show()

+-------------------------+
|total_prohibited_under_16|
+-------------------------+
|                      571|
+-------------------------+



In [0]:
spark.sql("""
          SELECT * FROM required_age_table
          WHERE required_age == 180
""").show()

+------------------+------------+
|              name|required_age|
+------------------+------------+
| Kissing Simulator|         180|
|Internet Simulator|         180|
|   Piano Simulator|         180|
|    ЕСТЬ ДВА СТУЛА|         180|
+------------------+------------+



## Analyse des genres

#### 7. Quels sont les genres les plus représentés ?
--> Le genre Indie est le plus représenté avec 39681 jeux soit plus de 60%

In [0]:
selected_columns = ['name', 'publisher', 'genre', 'positive', 'negative', 'initialprice', 'discount', 'price', 'owners']
df_genre_initial = df_clean.select(selected_columns)
df_genre_initial.printSchema()

root
 |-- name: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- positive: long (nullable = true)
 |-- negative: long (nullable = true)
 |-- initialprice: double (nullable = true)
 |-- discount: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- owners: string (nullable = true)



In [0]:
from pyspark.sql.functions import split, col
df_genre_list = df_genre_initial.withColumn("genre", split(col("genre"),","))
df_genre_list.show()

+------------------------------------+----------------------------+--------------------+--------+--------+------------+--------+-----+--------------------+
|                                name|                   publisher|               genre|positive|negative|initialprice|discount|price|              owners|
+------------------------------------+----------------------------+--------------------+--------+--------+------------+--------+-----+--------------------+
|                      Counter-Strike|                       Valve|            [Action]|  201215|    5199|        9.99|       0| 9.99|10,000,000 .. 20,...|
|                           ASCENXION|        PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|         0 .. 20,000|
|                         Crown Trick|        Team17, NEXT Studios|[Adventure,  Indi...|    4032|     646|       19.99|      70| 5.99|  200,000 .. 500,000|
|                Cook, Serve, Deli...|         Vertigo Gaming In

In [0]:
df_genre_list.printSchema()

root
 |-- name: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- genre: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- positive: long (nullable = true)
 |-- negative: long (nullable = true)
 |-- initialprice: double (nullable = true)
 |-- discount: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- owners: string (nullable = true)



In [0]:
from pyspark.sql.functions import explode
df_genre = df_genre_list.withColumn('genre_exploded', explode(df_genre_list.genre))
df_genre.show()

+--------------------+--------------------+--------------------+--------+--------+------------+--------+-----+--------------------+--------------+
|                name|           publisher|               genre|positive|negative|initialprice|discount|price|              owners|genre_exploded|
+--------------------+--------------------+--------------------+--------+--------+------------+--------+-----+--------------------+--------------+
|      Counter-Strike|               Valve|            [Action]|  201215|    5199|        9.99|       0| 9.99|10,000,000 .. 20,...|        Action|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|         0 .. 20,000|        Action|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|         0 .. 20,000|     Adventure|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|        

In [0]:
def sanitizeGenre(s: str) -> str:
        return s.replace(" ", "")

In [0]:
from pyspark.sql.functions import col, udf
sanitize_udf = udf(lambda x:sanitizeGenre(x))
df_genre_clean = df_genre.withColumn('genre_exploded', sanitize_udf(col('genre_exploded')))
df_genre_clean.printSchema()

root
 |-- name: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- genre: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- positive: long (nullable = true)
 |-- negative: long (nullable = true)
 |-- initialprice: double (nullable = true)
 |-- discount: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- owners: string (nullable = true)
 |-- genre_exploded: string (nullable = true)



In [0]:
df_genre_clean.createOrReplaceTempView('genre_table')

In [0]:
df_genre_clean.select('genre_exploded').distinct().count()

Out[81]: 29

In [0]:
df_genre_clean.select('genre_exploded').distinct().sort('genre_exploded', ascending=True).show(29)

+--------------------+
|      genre_exploded|
+--------------------+
|                    |
|          Accounting|
|              Action|
|           Adventure|
|  Animation&Modeling|
|     AudioProduction|
|              Casual|
| Design&Illustration|
|         EarlyAccess|
|           Education|
|          FreetoPlay|
|     GameDevelopment|
|                Gore|
|               Indie|
|MassivelyMultiplayer|
|               Movie|
|              Nudity|
|        PhotoEditing|
|                 RPG|
|              Racing|
|       SexualContent|
|          Simulation|
|    SoftwareTraining|
|              Sports|
|            Strategy|
|           Utilities|
|     VideoProduction|
|             Violent|
|       WebPublishing|
+--------------------+



In [0]:
spark.sql("""
          SELECT genre_exploded, COUNT(genre_exploded) AS cnt
          FROM genre_table
          GROUP BY genre_exploded
          ORDER BY cnt DESC
""").show()

+--------------------+-----+
|      genre_exploded|  cnt|
+--------------------+-----+
|               Indie|39681|
|              Action|23759|
|              Casual|22086|
|           Adventure|21431|
|            Strategy|10895|
|          Simulation|10836|
|                 RPG| 9534|
|         EarlyAccess| 6145|
|          FreetoPlay| 3393|
|              Sports| 2666|
|              Racing| 2155|
|MassivelyMultiplayer| 1460|
|           Utilities|  682|
| Design&Illustration|  406|
|  Animation&Modeling|  322|
|           Education|  317|
|     VideoProduction|  247|
|     AudioProduction|  195|
|             Violent|  168|
|    SoftwareTraining|  164|
+--------------------+-----+
only showing top 20 rows



#### 8. Existe-t-il des genres qui ont un meilleur ratio d'avis positifs/négatifs ?
--> Les genres Game Development, Adventure, Casual et RPG possèdent le meilleur ratio moyen de 6

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import round
df_genre_review = df_genre_clean.withColumn('review_ratio', round(col('positive')/col('negative'),2))
df_genre_review.show()
df_genre_review.printSchema()

+--------------------+--------------------+--------------------+--------+--------+------------+--------+-----+--------------------+--------------+------------+
|                name|           publisher|               genre|positive|negative|initialprice|discount|price|              owners|genre_exploded|review_ratio|
+--------------------+--------------------+--------------------+--------+--------+------------+--------+-----+--------------------+--------------+------------+
|      Counter-Strike|               Valve|            [Action]|  201215|    5199|        9.99|       0| 9.99|10,000,000 .. 20,...|        Action|        38.7|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|         0 .. 20,000|        Action|         5.4|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|         0 .. 20,000|     Adventure|         5.4|
|           ASCENXION|PsychoFlux Entert.

In [0]:
df_genre_review.createOrReplaceTempView('genre_review_table')
spark.sql("""
          SELECT genre_exploded, SUM(positive) AS sum_positive, SUM(negative) AS sum_negative, ROUND(sum_positive/sum_negative,2) AS ratio
          FROM genre_review_table
          GROUP BY genre_exploded
          ORDER BY ratio DESC
""").show(truncate=False)

+-------------------+------------+------------+-----+
|genre_exploded     |sum_positive|sum_negative|ratio|
+-------------------+------------+------------+-----+
|PhotoEditing       |577751      |13745       |42.03|
|Animation&Modeling |690765      |26392       |26.17|
|Design&Illustration|674057      |27007       |24.96|
|Utilities          |739335      |43503       |17.0 |
|GameDevelopment    |27461       |3274        |8.39 |
|Indie              |32531023    |4241234     |7.67 |
|AudioProduction    |69118       |9428        |7.33 |
|                   |183846      |25973       |7.08 |
|VideoProduction    |111514      |16362       |6.82 |
|Casual             |10034967    |1537296     |6.53 |
|WebPublishing      |33984       |5238        |6.49 |
|Simulation         |15572390    |2400512     |6.49 |
|Racing             |2340353     |383691      |6.1  |
|RPG                |19425528    |3274328     |5.93 |
|Action             |54858618    |9687659     |5.66 |
|Strategy           |1340287

In [0]:
spark.sql("""
          SELECT genre_exploded, ROUND(AVG(review_ratio),2) as avg_ratio
          FROM genre_review_table
          GROUP BY genre_exploded
          ORDER BY avg_ratio DESC
""").show(truncate=False)

+-------------------+---------+
|genre_exploded     |avg_ratio|
+-------------------+---------+
|GameDevelopment    |6.58     |
|Adventure          |6.24     |
|Casual             |6.05     |
|Indie              |5.96     |
|RPG                |5.64     |
|Animation&Modeling |5.51     |
|Design&Illustration|5.39     |
|Action             |5.38     |
|FreetoPlay         |5.29     |
|Simulation         |4.86     |
|PhotoEditing       |4.85     |
|                   |4.79     |
|Strategy           |4.78     |
|EarlyAccess        |4.69     |
|WebPublishing      |4.65     |
|Racing             |4.4      |
|Sports             |4.33     |
|Utilities          |4.28     |
|VideoProduction    |3.99     |
|Nudity             |3.92     |
+-------------------+---------+
only showing top 20 rows



#### 9. Certains éditeurs ont-ils des genres favoris ?

--> Voici les genres favoris des plus grands éditeurs :

- Big Fish : Casual, Adventure
- 8Floor : Casual
- SEGA : Action
- Square Enix : Action
- Choice of Games : RPG, Indie, Adventure
- Sekai Project : Casual, Indie, Adventure
- HH-Games : Casual

In [0]:
df_genre_clean.createOrReplaceTempView("genre_table")

spark.sql("""
          SELECT publisher, genre_exploded, COUNT(publisher) as cnt
          FROM genre_table
          GROUP BY publisher, genre_exploded
          ORDER BY cnt DESC, publisher
""").show(20, truncate=False)

+--------------------+--------------+---+
|publisher           |genre_exploded|cnt|
+--------------------+--------------+---+
|Big Fish Games      |Casual        |418|
|Big Fish Games      |Adventure     |392|
|8floor              |Casual        |202|
|Choice of Games     |RPG           |139|
|Choice of Games     |Indie         |136|
|HH-Games            |Casual        |132|
|Laush Studio        |Indie         |124|
|Choice of Games     |Adventure     |112|
|                    |Indie         |106|
|Alawar Entertainment|Casual        |105|
|Sekai Project       |Casual        |99 |
|Sokpop Collective   |Indie         |97 |
|Slitherine Ltd.     |Strategy      |96 |
|Alawar Entertainment|Adventure     |95 |
|Reforged Group      |Indie         |88 |
|Sekai Project       |Indie         |88 |
|Devolver Digital    |Indie         |87 |
|Laush Studio        |Casual        |87 |
|PLAYISM             |Indie         |81 |
|SEGA                |Action        |80 |
+--------------------+------------

#### 10. Quels sont les genres les plus lucratifs ?

--> Les genres Action, Adventure et Indie sont les plus lucratifs, ils génèrent respectivement :  
  - un chiffre d'affaires minimum de 35 milliard, 22 millard et 19 milliard de dollars (fourchette basse)
  - un chiffre d'affaires maximum de 80 milliard, 51 milliard et 45 milliard de dollars (fourchette haute)

In [0]:
df_genre_clean.select('owners').distinct().count()

Out[88]: 13

In [0]:
df_genre_clean.select('owners').distinct().sort('owners', ascending=False).show(13,truncate=False)

+--------------------------+
|owners                    |
+--------------------------+
|500,000 .. 1,000,000      |
|50,000,000 .. 100,000,000 |
|50,000 .. 100,000         |
|5,000,000 .. 10,000,000   |
|200,000,000 .. 500,000,000|
|200,000 .. 500,000        |
|20,000,000 .. 50,000,000  |
|20,000 .. 50,000          |
|2,000,000 .. 5,000,000    |
|100,000 .. 200,000        |
|10,000,000 .. 20,000,000  |
|1,000,000 .. 2,000,000    |
|0 .. 20,000               |
+--------------------------+



In [0]:
def extractMinOwners(str):
    result = str.split(" .. ")
    return int(''.join(result[0].split(",")))

def extractMaxOwners(str):
    result = str.split(" .. ")
    return int(''.join(result[1].split(",")))

In [0]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

to_int_udf_1 = udf(lambda x:extractMinOwners(x),IntegerType())
to_int_udf_2 = udf(lambda x:extractMaxOwners(x),IntegerType())

df_genre_revenue = df_genre_clean \
                    .withColumn('min_owners', to_int_udf_1(col('owners'))) \
                    .withColumn('max_owners', to_int_udf_2(col('owners')))

In [0]:
df_genre_revenue.printSchema()

root
 |-- name: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- genre: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- positive: long (nullable = true)
 |-- negative: long (nullable = true)
 |-- initialprice: double (nullable = true)
 |-- discount: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- owners: string (nullable = true)
 |-- genre_exploded: string (nullable = true)
 |-- min_owners: integer (nullable = true)
 |-- max_owners: integer (nullable = true)



In [0]:
df_genre_revenue.show()

+--------------------+--------------------+--------------------+--------+--------+------------+--------+-----+--------------------+--------------+----------+----------+
|                name|           publisher|               genre|positive|negative|initialprice|discount|price|              owners|genre_exploded|min_owners|max_owners|
+--------------------+--------------------+--------------------+--------+--------+------------+--------+-----+--------------------+--------------+----------+----------+
|      Counter-Strike|               Valve|            [Action]|  201215|    5199|        9.99|       0| 9.99|10,000,000 .. 20,...|        Action|  10000000|  20000000|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|         0 .. 20,000|        Action|         0|     20000|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|         0 .. 20,000|     Adventure|         

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

df_genre_revenue = df_genre_revenue \
                        .withColumn('min_revenue', col('min_owners')*col('price').cast('float')) \
                        .withColumn('max_revenue', col('max_owners')*col('price').cast('float')) \
                        .drop('owners')
                        
df_genre_revenue.printSchema()

root
 |-- name: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- genre: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- positive: long (nullable = true)
 |-- negative: long (nullable = true)
 |-- initialprice: double (nullable = true)
 |-- discount: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- genre_exploded: string (nullable = true)
 |-- min_owners: integer (nullable = true)
 |-- max_owners: integer (nullable = true)
 |-- min_revenue: float (nullable = true)
 |-- max_revenue: float (nullable = true)



In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

df_revenue = df_genre_revenue \
                        .withColumn('min_revenue', col('min_revenue').cast(IntegerType())) \
                        .withColumn('max_revenue', col('max_revenue').cast(IntegerType()))

df_revenue.show()

+--------------------+--------------------+--------------------+--------+--------+------------+--------+-----+--------------+----------+----------+-----------+-----------+
|                name|           publisher|               genre|positive|negative|initialprice|discount|price|genre_exploded|min_owners|max_owners|min_revenue|max_revenue|
+--------------------+--------------------+--------------------+--------+--------+------------+--------+-----+--------------+----------+----------+-----------+-----------+
|      Counter-Strike|               Valve|            [Action]|  201215|    5199|        9.99|       0| 9.99|        Action|  10000000|  20000000|   99900000|  199800000|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|        Action|         0|     20000|          0|     199800|
|           ASCENXION|PsychoFlux Entert...|[Action,  Adventu...|      27|       5|        9.99|       0| 9.99|     Adventure|         0|    

In [0]:
df_revenue.createOrReplaceTempView('revenue_table')
spark.sql("""
          SELECT genre_exploded, 
            SUM(min_revenue) AS total_min_revenue, 
            SUM(max_revenue) AS total_max_revenue
          FROM revenue_table
          GROUP BY genre_exploded
          ORDER BY total_min_revenue DESC, total_max_revenue DESC
""").show(30, truncate=False)

+--------------------+-----------------+-----------------+
|genre_exploded      |total_min_revenue|total_max_revenue|
+--------------------+-----------------+-----------------+
|Action              |35929269657      |80731619140      |
|Adventure           |22618905951      |51872567919      |
|Indie               |19134715838      |45558431538      |
|RPG                 |16675128197      |36819140786      |
|Strategy            |12362391808      |27937689117      |
|Simulation          |11422233094      |26117264826      |
|Casual              |4476567857       |11685341454      |
|MassivelyMultiplayer|3692721317       |8167594157       |
|EarlyAccess         |3147066715       |7770255341       |
|Sports              |1816053347       |4483740991       |
|Racing              |1579747645       |3867214039       |
|Utilities           |190300698        |567310353        |
|Design&Illustration |154659699        |499962076        |
|                    |148135094        |306695689       

## Analyse des plateformes

#### 11. La plupart des jeux sont-ils plutôt disponibles sur Windows/Mac/Linux ?

--> Presque tous les jeux tournent sur Windows (55676 soit 99,97% des jeux)

In [0]:
selected_columns = ['name', 'platforms_linux', 'platforms_mac', 'platforms_windows']
df_platform = df_clean.select(selected_columns)
df_platform.printSchema()
df_platform.show()

root
 |-- name: string (nullable = true)
 |-- platforms_linux: boolean (nullable = true)
 |-- platforms_mac: boolean (nullable = true)
 |-- platforms_windows: boolean (nullable = true)

+------------------------------------+---------------+-------------+-----------------+
|                                name|platforms_linux|platforms_mac|platforms_windows|
+------------------------------------+---------------+-------------+-----------------+
|                      Counter-Strike|           true|         true|             true|
|                           ASCENXION|          false|        false|             true|
|                         Crown Trick|          false|        false|             true|
|                Cook, Serve, Deli...|          false|         true|             true|
|                            细胞战争|          false|        false|             true|
|                             Zengeon|          false|         true|             true|
|干支セトラ　陽ノ卷｜干支etc.　陽之卷|          fal

In [0]:
linux = df_platform.select('platforms_linux').where(df_platform['platforms_linux'] == True).count()
mac = df_platform.select('platforms_mac').where(df_platform['platforms_mac'] == True).count()
windows = df_platform.select('platforms_windows').where(df_platform['platforms_windows'] == True).count()

print(f'linux: {linux}, mac: {mac}, windows: {windows}')

linux: 8457, mac: 12769, windows: 55675


#### 12. Certains genres ont-ils tendance à être disponibles préférentiellement sur certaines plateformes ?

--> La distribution des plateformes est plutôt uniforme pour tous les genres, sauf pour :   
- Les jeux en lien avec les films / le cinéma : seulement sur Windows  
- Les jeux en lien avec la comptabilité : sur Windows et Mac, absent sur Linux  
- Les jeux en lien avec la création photo, vidéo et audio : majoritairement sur Windows et Mac, très peu sur Linux
- Les jeux en lien avec l'éducation et la formation : majoritairement sur Windows et Mac, très peu sur Linux

In [0]:
selected_columns = ['name', 'genre', 'platforms_linux', 'platforms_mac', 'platforms_windows']
df_genre_and_platforms = df_clean.select(selected_columns)
df_genre_and_platforms.printSchema()

root
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- platforms_linux: boolean (nullable = true)
 |-- platforms_mac: boolean (nullable = true)
 |-- platforms_windows: boolean (nullable = true)



In [0]:
from pyspark.sql.functions import split, col
df_genre_liste = df_genre_and_platforms.withColumn("genre", split(col("genre"),","))
df_genre_liste.show()

+------------------------------------+--------------------+---------------+-------------+-----------------+
|                                name|               genre|platforms_linux|platforms_mac|platforms_windows|
+------------------------------------+--------------------+---------------+-------------+-----------------+
|                      Counter-Strike|            [Action]|           true|         true|             true|
|                           ASCENXION|[Action,  Adventu...|          false|        false|             true|
|                         Crown Trick|[Adventure,  Indi...|          false|        false|             true|
|                Cook, Serve, Deli...|[Action,  Indie, ...|          false|         true|             true|
|                            细胞战争|[Action,  Casual,...|          false|        false|             true|
|                             Zengeon|[Action,  Adventu...|          false|         true|             true|
|干支セトラ　陽ノ卷｜干支etc.　陽之卷|[Adventure

In [0]:
from pyspark.sql.functions import explode
df_genre_final = df_genre_liste.withColumn('genre_exploded', explode(df_genre_liste.genre))
df_genre_final.show()

+--------------------+--------------------+---------------+-------------+-----------------+--------------+
|                name|               genre|platforms_linux|platforms_mac|platforms_windows|genre_exploded|
+--------------------+--------------------+---------------+-------------+-----------------+--------------+
|      Counter-Strike|            [Action]|           true|         true|             true|        Action|
|           ASCENXION|[Action,  Adventu...|          false|        false|             true|        Action|
|           ASCENXION|[Action,  Adventu...|          false|        false|             true|     Adventure|
|           ASCENXION|[Action,  Adventu...|          false|        false|             true|         Indie|
|         Crown Trick|[Adventure,  Indi...|          false|        false|             true|     Adventure|
|         Crown Trick|[Adventure,  Indi...|          false|        false|             true|         Indie|
|         Crown Trick|[Adventure,  In

In [0]:
def sanitizeGenre(s: str) -> str:
        return s.replace(" ", "")
    
from pyspark.sql.functions import col, udf
sanitize_udf = udf(lambda x:sanitizeGenre(x))
df_genre_final_clean = df_genre_final.withColumn('genre_exploded', sanitize_udf(col('genre_exploded')))

In [0]:
df_genre_final_clean.createOrReplaceTempView('genre_and_platforms_table')
spark.sql("""
          SELECT genre_exploded, 
            COUNT(CASE WHEN platforms_linux = true THEN 1 END) AS linux,
            COUNT(CASE WHEN platforms_mac = true THEN 1 END) AS mac,
            COUNT(CASE WHEN platforms_windows = true THEN 1 END) AS windows
          FROM genre_and_platforms_table
          GROUP BY genre_exploded
          ORDER BY windows DESC, mac DESC, linux DESC
""").show(30)

+--------------------+-----+----+-------+
|      genre_exploded|linux| mac|windows|
+--------------------+-----+----+-------+
|               Indie| 6978|9935|  39676|
|              Action| 3379|4564|  23755|
|              Casual| 3305|5130|  22082|
|           Adventure| 3302|5039|  21427|
|            Strategy| 1826|3005|  10892|
|          Simulation| 1532|2439|  10832|
|                 RPG| 1524|2248|   9533|
|         EarlyAccess|  632| 900|   6145|
|          FreetoPlay|  474| 845|   3391|
|              Sports|  287| 506|   2665|
|              Racing|  304| 424|   2154|
|MassivelyMultiplayer|  164| 270|   1459|
|           Utilities|   49| 102|    681|
| Design&Illustration|   54| 100|    405|
|  Animation&Modeling|   38|  74|    322|
|           Education|   19|  56|    317|
|     VideoProduction|    6|  29|    247|
|     AudioProduction|    7|  41|    193|
|             Violent|   22|  33|    168|
|    SoftwareTraining|   12|  27|    164|
|     GameDevelopment|   35|  52| 

In [0]:
data = spark.sql("""
          SELECT genre_exploded, 
            COUNT(CASE WHEN platforms_linux = true THEN 1 END) AS linux,
            COUNT(CASE WHEN platforms_mac = true THEN 1 END) AS mac,
            COUNT(CASE WHEN platforms_windows = true THEN 1 END) AS windows
          FROM genre_and_platforms_table
          GROUP BY genre_exploded
          ORDER BY windows DESC, mac DESC, linux DESC
""")

In [0]:
# Ouvrez le notebook dans Databrick pour voir la visualisation :
display(data)

genre_exploded,linux,mac,windows
Indie,6978,9935,39676
Action,3379,4564,23755
Casual,3305,5130,22082
Adventure,3302,5039,21427
Strategy,1826,3005,10892
Simulation,1532,2439,10832
RPG,1524,2248,9533
EarlyAccess,632,900,6145
FreetoPlay,474,845,3391
Sports,287,506,2665


Databricks visualization. Run in Databricks to view.

## Analyse des jeux les plus populaires

#### 13. Quelles sont les caractéristiques des jeux les plus joués ?

- Il y a un seul jeu qui cumule plus de 200 millions de téléchargements et il est gratuit  
- Il y a 4 jeux qui cumulent plus de 50 millions de téléchargements et ils sont au prix moyen de 4,99$  
- Il y a 21 jeux qui cumulent plus de 20 millions de téléchargements et ils sont au prix moyen de 10$  
- La majorité des jeux (38072) ont été téléchargé moins de 20000 fois et ils sont au prix moyen de 7$  
- Si on veut performer en nombre de téléchargement on devrait pas mettre un prix de plus de 4,99$  

In [0]:
df_genre_list.createOrReplaceTempView('price_and_owners_table')

spark.sql("""
          SELECT owners, COUNT(owners) AS owners_group, AVG(price)
          FROM price_and_owners_table
          GROUP BY owners
          ORDER BY owners_group DESC
""").show()

+--------------------+------------+------------------+
|              owners|owners_group|        avg(price)|
+--------------------+------------+------------------+
|         0 .. 20,000|       38072| 7.097593507035998|
|    20,000 .. 50,000|        7285| 7.300844200411549|
|   50,000 .. 100,000|        3695| 8.340476319350978|
|  100,000 .. 200,000|        2519|  9.32903930131028|
|  200,000 .. 500,000|        2162|10.661660499537653|
|500,000 .. 1,000,000|         932| 13.94149141630887|
|1,000,000 .. 2,00...|         526|15.781977186311666|
|2,000,000 .. 5,00...|         335|18.133731343283475|
|5,000,000 .. 10,0...|          97|18.260618556701036|
|10,000,000 .. 20,...|          41| 12.09146341463415|
|20,000,000 .. 50,...|          21|10.422857142857145|
|50,000,000 .. 100...|           4|            4.9975|
|200,000,000 .. 50...|           1|               0.0|
+--------------------+------------+------------------+



- Les gens sont prêts à payer jusqu'à 59$ pour un bon jeu
- Dans les 4 jeux à plus de 50 milliers de téléchargements il y en a un à 39$, le reste sont gratuits
- Dans les 21 jeux à plus de 20 millions de téléchargement, la plupart sont gratuits
- Il y a quelques jeux à 4, 5, 10, 20, 30, 40 et 60$

In [0]:
df_clean.createOrReplaceTempView("clean_table")
spark.sql("""
          SELECT name, publisher, genre, owners, initialprice, discount, price, release_date, required_age
          FROM clean_table
          WHERE (owners = '200,000,000 .. 500,000,000' OR owners = '50,000,000 .. 100,000,000' OR owners = '20,000,000 .. 50,000,000')
          ORDER BY owners DESC
""").show(30)

+--------------------+--------------------+--------------------+--------------------+------------+--------+-----+------------+------------+
|                name|           publisher|               genre|              owners|initialprice|discount|price|release_date|required_age|
+--------------------+--------------------+--------------------+--------------------+------------+--------+-----+------------+------------+
|           New World|        Amazon Games|Action, Adventure...|50,000,000 .. 100...|       39.99|      50|19.99|  2021/09/28|           0|
|     Team Fortress 2|               Valve|Action, Free to Play|50,000,000 .. 100...|         0.0|       0|  0.0|  2007/10/10|           0|
| PUBG: BATTLEGROUNDS|       KRAFTON, Inc.|Action, Adventure...|50,000,000 .. 100...|         0.0|       0|  0.0|  2017/12/21|           0|
|Counter-Strike: G...|               Valve|Action, Free to Play|50,000,000 .. 100...|         0.0|       0|  0.0|  2012/08/21|           0|
|              Dota 

# Conclusion

- Les jeux qui marchent bien sont principalement sur Windows et Mac
- Les genres les plus joués sont dans l'ordre : Indie, Action, Casual, Adventure
- La fourchette de prix moyenne : de 0 à 10 dollars
- Pour vendre plus il vaut mieux proposer un prix entre 0 et 4,99 dollars
- Il faut absolument avoir une version anglaise
- Et proposer un jeu tout public sans restriction d'âge